# Imports

In [1]:
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model, load_model
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

Using TensorFlow backend.


# Read Data

In [2]:
# variablesIndex = [5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
# variablesName = ["T", "H", "H2", "O", "O2", "OH", "H2O", "HO2", "H2O2", "CO", "CO2", "CH2O",
#                  "HO2CHO", "O2CHO", "CH3O2H", "CH3O2", "CH4", "CH3", "C2H5", "C2H4", "C2H3",
#                  "CH3CHO", "C2H5OH", "O2C2H4OH"]

variablesIndex = [5, 8, 9, 10, 11, 12, 13, 16, 17, 23, 29, 30]
variablesName = ['T', 'H', 'H2', 'O', 'O2', 'OH', 'H2O', 'CO', 'CO2', 'CH4', 'C2H5OH', 'O2C2H4OH']

In [3]:
clean_data = []
for j in range(len(variablesIndex)):
  start = time.time()
  clean_data.append(pd.read_csv(
    'drive/My Drive/DNS Autoencoders/plane-xy-0.50.2.200E-03.csv',
    header=None,
    index_col=None,
    skiprows=lambda row: row < (31360 * (variablesIndex[j] - 1) + 37),
    nrows=31360,
    engine='python',
    dtype='float32'
  ))
  stop = time.time()
  print(f'Variable {variablesName[j]} Read in {round(stop-start, 2)}s')

Variable T Read in 4.73s
Variable H Read in 0.69s
Variable H2 Read in 0.72s
Variable O Read in 0.9s
Variable O2 Read in 0.86s
Variable OH Read in 0.95s
Variable H2O Read in 1.1s
Variable CO Read in 1.19s
Variable CO2 Read in 1.26s
Variable CH4 Read in 1.77s
Variable C2H5OH Read in 2.02s
Variable O2C2H4OH Read in 2.07s


# Scale + Shape Data

In [4]:
mins, maxs = [], []
for j in range(len(variablesIndex)):
    mins.append(clean_data[j].to_numpy().min())
    maxs.append(clean_data[j].to_numpy().max())
    clean_data[j] = (clean_data[j] - np.full_like(clean_data[j], mins[j])) / (np.full_like(clean_data[j], maxs[j]) - np.full_like(clean_data[j], mins[j]))

In [5]:
for j in range(len(variablesIndex)):
    clean_data[j] = np.array(clean_data[j]).reshape((560, 560))

# Create Noisy Data

In [6]:
noisy_data = list(clean_data)
for j in range(len(variablesIndex)):
    noisy_data[j] = noisy_data[j] + 0.05 * np.random.normal(loc=0.0, scale=1.0, size=noisy_data[j].shape)

# Stack Images for Training + Test

In [7]:
# stack clean and noisy training images together
clean_stacked_train = np.concatenate(clean_data).reshape((1, len(variablesIndex)*560, 560, 1))
noisy_stacked_train = np.concatenate(noisy_data).reshape((1, len(variablesIndex)*560, 560, 1))

# Build Autoencoder

In [8]:
input_img = Input(shape=(len(variablesIndex)*560, 560, 1))

x = Conv2D(64, 3, padding='same', activation='relu')(input_img)
x = MaxPooling2D(pool_size=2)(x)
encoder = Conv2D(64, 3, padding='same', activation='relu')(x)

x = Conv2DTranspose(64, 3, padding='same', activation='relu')(encoder)
x = UpSampling2D(size=2)(x)
decoder = Conv2DTranspose(64, 3, padding='same', activation='relu')(x)

output_img = Conv2D(1, 1, padding='same', activation='sigmoid')(decoder)

In [9]:
autoencoder = Model(input_img, output_img)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6720, 560, 1)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6720, 560, 64)     640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3360, 280, 64)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3360, 280, 64)     36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 3360, 280, 64)     36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 6720, 560, 64)     0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 6720, 560, 64)     3692

# Fit Autoencoder + Load Best Model

In [10]:
autoencoder.compile('adam', loss='binary_crossentropy')
autoencoder.fit(
    noisy_stacked_train, 
    clean_stacked_train,
    batch_size=None,
    epochs=50,
    callbacks=[
      ModelCheckpoint('drive/My Drive/DNS Autoencoders/bestModel_2.200E-03', monitor='loss', save_best_only=True), 
      EarlyStopping(monitor='loss', patience=3)
    ]
)

Epoch 1/50
1/1 [==============================] - 9s 9s/step - loss: 0.6958
Epoch 2/50
1/1 [==============================] - 1s 1s/step - loss: 0.6877
Epoch 3/50
1/1 [==============================] - 1s 1s/step - loss: 0.6791
Epoch 4/50
1/1 [==============================] - 1s 1s/step - loss: 0.6679
Epoch 5/50
1/1 [==============================] - 1s 1s/step - loss: 0.6533
Epoch 6/50
1/1 [==============================] - 1s 1s/step - loss: 0.6351
Epoch 7/50
1/1 [==============================] - 1s 1s/step - loss: 0.6118
Epoch 8/50
1/1 [==============================] - 1s 1s/step - loss: 0.5819
Epoch 9/50
1/1 [==============================] - 1s 1s/step - loss: 0.5444
Epoch 10/50
1/1 [==============================] - 1s 1s/step - loss: 0.5003
Epoch 11/50
1/1 [==============================] - 1s 1s/step - loss: 0.4656
Epoch 12/50
1/1 [==============================] - 1s 1s/step - loss: 0.4347
Epoch 13/50
1/1 [==============================] - 1s 1s/step - loss: 0.4097
Epoch 14

In [11]:
best_autoencoder = load_model('drive/My Drive/DNS Autoencoders/bestModel_2.200E-03')

# Predict + Visualize on Training Set

In [12]:
predicted_train = np.split(best_autoencoder.predict_on_batch(noisy_stacked_train).reshape((len(variablesIndex)*560, 560)), len(variablesIndex))

In [13]:
for j in range(len(variablesIndex)):
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 10))
  fig.suptitle(f'Variable = {variablesName[j]}', y=0.8)
  
  ax1.set_title("Clean")
  ax1.imshow(clean_data[j])
  
  ax2.set_title("Noisy")
  ax2.imshow(noisy_data[j])

  ax3.set_title("Predicted")
  ax3.imshow(predicted_train[j])

  plt.tight_layout()
  plt.savefig(f'drive/My Drive/DNS Autoencoders/2.200E-03 Results/{variablesName[j]}.tiff')

  plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Inverse Scale the Data

In [14]:
for j in range(len(variablesIndex)):
  predicted_train[j] = predicted_train[j]*(np.full_like(predicted_train[j], maxs[j]) - np.full_like(predicted_train[j], mins[j])) + np.full_like(predicted_train[j], mins[j])
  clean_data[j] =clean_data[j]*(np.full_like(clean_data[j], maxs[j]) - np.full_like(clean_data[j], mins[j])) + np.full_like(clean_data[j], mins[j])
  noisy_data[j] =noisy_data[j]*(np.full_like(noisy_data[j], maxs[j]) - np.full_like(noisy_data[j], mins[j])) + np.full_like(noisy_data[j], mins[j])

In [15]:
pd.options.display.float_format = '{:.2f}'.format
for j in range(len(variablesIndex)):
  print(f'Variable = {variablesName[j]}')
  display(pd.DataFrame(clean_data[j]).head())
  display(pd.DataFrame(noisy_data[j]).head())
  display(pd.DataFrame(predicted_train[j]).head())

Variable = T


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,1690.00,1690.00,1690.00,1680.00,1680.00,1680.00,1680.00,1670.00,1670.00,1670.00,1660.00,1660.00,1650.00,1650.00,1640.00,1640.00,1630.00,1630.00,1620.00,1620.00,1610.00,1610.00,1600.00,1600.00,1590.00,1590.00,1580.00,1580.00,1570.00,1570.00,1570.00,1560.00,1560.00,1550.00,1550.00,1550.00,1550.00,1540.00,1540.00,1540.00,...,1710.00,1710.00,1710.00,1710.00,1710.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1690.00,1690.00
1,1690.00,1690.00,1690.00,1690.00,1680.00,1680.00,1680.00,1680.00,1670.00,1670.00,1670.00,1660.00,1660.00,1650.00,1650.00,1650.00,1640.00,1640.00,1630.00,1630.00,1620.00,1620.00,1610.00,1610.00,1600.00,1600.00,1590.00,1590.00,1580.00,1580.00,1570.00,1570.00,1560.00,1560.00,1560.00,1550.00,1550.00,1550.00,1540.00,1540.00,...,1710.00,1710.00,1710.00,1710.00,1710.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1690.00,1690.00
2,1690.00,1690.00,1690.00,1690.00,1690.00,1680.00,1680.00,1680.00,1680.00,1670.00,1670.00,1670.00,1660.00,1660.00,1660.00,1650.00,1650.00,1640.00,1640.00,1630.00,1630.00,1620.00,1620.00,1610.00,1610.00,1600.00,1600.00,1590.00,1590.00,1580.00,1580.00,1570.00,1570.00,1560.00,1560.00,1560.00,1550.00,1550.00,1550.00,1540.00,...,1710.00,1710.00,1710.00,1710.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1690.00,1690.00
3,1690.00,1690.00,1690.00,1690.00,1690.00,1680.00,1680.00,1680.00,1680.00,1680.00,1670.00,1670.00,1670.00,1660.00,1660.00,1660.00,1650.00,1650.00,1650.00,1640.00,1640.00,1630.00,1630.00,1620.00,1620.00,1610.00,1610.00,1600.00,1600.00,1590.00,1590.00,1580.00,1580.00,1570.00,1570.00,1560.00,1560.00,1550.00,1550.00,1550.00,...,1710.00,1710.00,1710.00,1710.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1690.00,1690.00
4,1690.00,1690.00,1690.00,1690.00,1690.00,1690.00,1680.00,1680.00,1680.00,1680.00,1680.00,1670.00,1670.00,1670.00,1660.00,1660.00,1660.00,1650.00,1650.00,1650.00,1640.00,1640.00,1630.00,1630.00,1620.00,1620.00,1620.00,1610.00,1610.00,1600.00,1600.00,1590.00,1590.00,1580.00,1580.00,1570.00,1570.00,1560.00,1560.00,1550.00,...,1710.00,1710.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1690.00,1690.00,1690.00


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,1685.59,1708.08,1691.99,1647.50,1667.89,1683.64,1720.38,1688.99,1713.27,1703.43,1689.39,1698.80,1634.88,1714.67,1624.62,1655.70,1634.36,1610.56,1578.55,1695.23,1596.62,1624.04,1555.72,1578.77,1608.44,1562.08,1553.46,1562.80,1529.61,1599.66,1569.26,1496.55,1535.01,1552.15,1467.19,1567.24,1599.16,1567.11,1540.95,1528.19,...,1705.21,1674.68,1727.80,1659.30,1695.88,1644.32,1733.43,1664.77,1708.14,1688.68,1680.35,1752.36,1714.50,1692.51,1662.09,1579.54,1631.50,1703.28,1664.70,1665.13,1672.26,1716.06,1687.83,1728.78,1728.22,1683.35,1662.58,1631.94,1669.75,1701.00,1729.10,1671.72,1696.26,1678.27,1682.90,1761.37,1699.54,1636.91,1810.29,1664.29
1,1680.01,1684.86,1742.02,1663.31,1705.86,1618.02,1677.43,1647.98,1662.23,1667.02,1692.37,1554.40,1695.48,1641.30,1719.44,1661.16,1606.01,1670.01,1596.63,1603.62,1603.69,1664.57,1603.85,1609.27,1599.18,1630.87,1617.18,1586.56,1554.01,1606.98,1608.12,1560.69,1524.19,1574.69,1596.47,1558.97,1580.32,1547.23,1468.48,1505.84,...,1687.60,1737.82,1685.30,1704.79,1685.64,1755.54,1666.16,1664.16,1771.62,1638.11,1664.23,1679.92,1687.49,1752.67,1660.32,1633.05,1660.66,1702.48,1716.81,1647.59,1685.90,1723.41,1767.20,1692.38,1682.12,1627.45,1735.86,1718.88,1771.85,1737.64,1704.71,1742.79,1649.96,1639.91,1659.59,1734.78,1715.59,1694.84,1675.36,1727.40
2,1729.54,1618.38,1660.03,1675.85,1767.35,1669.73,1691.60,1698.37,1664.56,1678.39,1712.60,1696.79,1693.36,1681.30,1636.32,1659.63,1674.59,1684.67,1614.96,1639.36,1686.23,1687.23,1583.92,1672.82,1655.91,1628.76,1616.93,1550.42,1604.33,1585.57,1624.52,1623.47,1535.07,1559.80,1533.56,1528.28,1534.00,1620.90,1565.66,1537.81,...,1684.03,1720.45,1700.27,1720.39,1778.34,1765.62,1726.04,1746.01,1695.52,1693.39,1693.22,1685.24,1701.92,1735.56,1680.11,1655.59,1689.90,1702.40,1638.17,1728.98,1680.88,1652.07,1734.17,1736.01,1679.41,1631.30,1726.94,1703.78,1744.90,1689.57,1723.60,1756.07,1749.54,1669.03,1722.44,1770.96,1726.93,1715.65,1702.54,1751.60
3,1685.16,1654.37,1750.51,1648.18,1683.06,1700.34,1657.95,1640.89,1710.99,1648.16,1698.38,1608.03,1660.10,1707.61,1620.87,1750.12,1669.82,1556.98,1621.04,1573.44,1684.81,1660.11,1641.68,1587.27,1632.30,1650.20,1651.13,1562.34,1596.04,1619.78,1564.78,1534.84,1577.25,1627.61,1606.13,1542.28,1511.19,1546.03,1521.55,1539.85,...,1713.51,1687.66,1713.76,1731.13,1709.89,1726.28,1750.73,1702.66,1758.14,1757.40,1740.46,1699.92,1711.18,1681.18,1691.37,1799.97,1683.66,1713.48,1708.50,1713.66,1720.61,1746.69,1710.49,1694.14,1712.51,1685.57,1701.51,1637.22,1685.83,1786.74,1727.88,1666.03,1724.25,1742.95,1718.39,1716.44,1711.87,1764.13,1642.90,1683.62
4,1732.69,1756.35,1637.26,1746.83,1714.58,1706.47,1693.83,1655.70,1657.30,1672.19,1677.98,1597.76,1704.86,1673.58,1742.61,1664.48,1661.72,1684.14,1678.34,1599.07,1652.30,1637.75,1581.38,1643.10,1630.92,1582.90,1600.66,1641.58,1613.12,1586.10,1672.80,1550.26,1561.82,1523.48,1568.84,1592.73,1484.82,1526.79,1497.42,1603.92,...,1746.55,1662.46,1670.44,1718.81,1718.93,1753.43,1706.09,1625.50,1682.39,1677.70,1747.96,1689.79,1712.43,1679.17,1708.05,1666.43,1685.66,1804.94,1679.17,1743.79,1760.69,1660.14,1634.15,1695.54,1715.77,1649.73,1781.35,1665.11,1670.54,1636.30,1707.56,1722.34,1721.99,1672.41,1690.64,1702.16,1633.55,1644.04,1659.78,1720.52


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,1341.68,1307.94,1340.25,1367.17,1381.32,1399.18,1406.36,1414.24,1414.50,1412.00,1409.01,1405.99,1406.13,1403.82,1401.26,1398.52,1398.16,1397.94,1396.51,1396.62,1398.04,1400.44,1400.94,1402.95,1403.88,1404.56,1403.53,1400.24,1397.54,1391.78,1387.32,1379.82,1375.07,1369.44,1368.70,1367.04,1364.61,1361.84,1360.22,1356.51,...,1425.66,1429.01,1431.11,1433.28,1433.42,1432.21,1431.15,1428.82,1427.70,1427.25,1427.48,1428.90,1429.84,1429.79,1428.74,1425.94,1423.89,1421.59,1421.33,1421.57,1421.20,1421.25,1422.22,1422.68,1422.35,1422.47,1422.83,1422.93,1423.24,1423.19,1423.56,1424.01,1423.48,1417.17,1409.80,1388.48,1375.64,1362.82,1369.63,1380.34
1,1342.22,1307.41,1366.20,1416.70,1445.68,1475.43,1487.50,1498.99,1497.74,1492.72,1488.63,1484.24,1483.82,1479.29,1475.30,1471.40,1469.48,1466.33,1463.83,1463.76,1464.32,1466.32,1466.92,1469.54,1470.78,1471.79,1469.44,1464.35,1459.67,1448.53,1440.03,1425.38,1415.99,1405.81,1402.58,1398.10,1395.35,1392.43,1390.48,1383.55,...,1521.63,1526.25,1529.89,1532.31,1531.98,1528.56,1525.90,1522.44,1521.01,1521.92,1523.36,1526.33,1527.19,1525.93,1524.25,1519.78,1516.33,1512.02,1509.99,1508.93,1509.81,1511.56,1512.92,1513.48,1513.17,1513.97,1514.67,1514.57,1515.02,1514.25,1513.62,1514.19,1513.57,1502.31,1490.36,1449.00,1418.06,1388.61,1385.24,1375.13
2,1372.35,1353.21,1433.69,1499.22,1532.40,1564.55,1574.91,1583.91,1582.14,1577.32,1573.97,1569.75,1568.69,1565.37,1562.04,1558.48,1556.31,1551.72,1548.67,1546.83,1546.29,1547.10,1546.73,1548.72,1549.71,1549.74,1546.84,1541.45,1536.71,1525.89,1517.73,1501.74,1491.21,1479.59,1474.85,1468.82,1467.29,1464.92,1461.98,1455.63,...,1606.42,1610.12,1612.85,1614.63,1614.26,1610.75,1608.39,1605.27,1604.56,1606.08,1607.45,1610.47,1611.30,1610.39,1608.48,1603.83,1600.92,1596.62,1594.28,1592.20,1593.64,1595.80,1596.88,1598.14,1598.31,1599.63,1599.72,1599.42,1599.54,1598.43,1598.22,1597.98,1596.92,1585.94,1576.15,1535.47,1499.82,1462.35,1443.88,1409.62
3,1437.39,1454.60,1552.95,1618.80,1649.02,1670.13,1676.79,1681.48,1679.37,1675.64,1673.34,1670.18,1668.90,1666.88,1665.35,1662.93,1660.94,1656.67,1653.72,1651.80,1650.48,1649.83,1648.47,1648.49,1648.83,1648.11,1645.92,1641.55,1638.42,1629.96,1622.36,1609.09,1600.34,1589.12,1583.17,1577.04,1576.27,1574.81,1572.62,1569.30,...,1696.02,1698.12,1698.75,1699.30,1699.27,1697.06,1695.51,1694.07,1694.02,1695.71,1697.37,1699.38,1699.88,1699.66,1698.00,1694.16,1691.81,1688.47,1686.81,1685.50,1685.77,1686.47,1687.87,1690.43,1690.93,1690.98,1691.36,1691.57,1690.41,1688.47,1688.32,1688.03,1687.74,1680.49,1673.41,1642.08,1609.84,1555.25,1517.02,1444.50
4,1467.70,1499.32,1598.70,1658.54,1683.80,1699.93,1704.24,1707.48,1705.49,1702.38,1700.42,1697.73,1696.84,1695.36,1694.54,1692.43,1690.57,1687.84,1685.90,1684.57,1683.10,1681.87,1680.53,1680.00,1679.83,1679.18,1677.50,1674.75,1672.50,1665.52,1659.12,1648.24,1641.51,1632.10,1626.28,1620.73,1619.79,1619.05,1617.85,1615.64,...,1717.86,1719.09,1719.38,1719.39,1719.25,1718.01,1716.95,1716.08,1716.06,1717.76,1719.16,1720.77,1721.13,1720.81,1719.54,1716.74,1715.02,1712.18,1711.12,1709.98,1709.70,1710.10,1711.42,1713.68,1714.04,1713.95,1714.33,1714.36,1713.42,1711.52,1711.17,1711.10,1711.14,1706.37,1700.89,1675.33,1646.91,1590.89,1547.44,1459.35


Variable = H


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Variable = O2C2H4OH


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,...,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00
1,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,...,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00
2,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,...,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00
3,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,...,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00
4,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,...,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
